# Part 2 of Capstone Project
## Clustering of Toronto Neighborhood

### Import Data from Wikipedia

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
#!pip install lxml

### Import Data from URL and to a Pandas-Dataframe
Extract Table of the Wiki-page with Beautifulsoup and send the table to a DataFrame


In [36]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

req=requests.get(url)
soup=BeautifulSoup(req.text,"html.parser")
table = soup.find_all('table')
df=pd.read_html(str(table))[0]

In [48]:
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df

Postal Code           Borough  \
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
5           M6A        North York   
6           M7A  Downtown Toronto   
..          ...               ...   
160         M8X         Etobicoke   
165         M4Y  Downtown Toronto   
168         M7Y      East Toronto   
169         M8Y         Etobicoke   
178         M8Z         Etobicoke   

                                         Neighbourhood  
2                                            Parkwoods  
3                                     Victoria Village  
4                            Regent Park, Harbourfront  
5                     Lawrence Manor, Lawrence Heights  
6          Queen's Park, Ontario Provincial Government  
..                                                 ...  
160      The Kingsway, Montgomery Road, Old Mill North  
165                               Church and Wellesley  
168  Business reply mail Processing Centre, South C...  
169  Old Mill South, King's Mill Park, Sunnylea, Hu...  
178  Mimico NW, The Queensway West, South of Bloor,...  

[103 rows x 3 columns]

### Check if all Neighbourhoods are filled and there is no "Not assigned"

In [49]:
not_assigned=0

for i, row in df.iterrows():
    if row[2] =='Not assigned':
        print(i,row[0],row[1],row[2])
        not_assigned=1
        
if not_assigned==0:
    print('Everything is fine!')

Everything is fine!


### Group Neighbourhood under same Post code

In [ ]:
df_group = df.groupby('Postal Code')['Neighbourhood'].apply(lambda x:"%s" % ', '.join(x))

In [ ]:
df_join= pd.merge(df, df_group, on='Postal Code')
df_join.drop(['Neighbourhood_x'], axis=1, inplace=True)
df_join.drop_duplicates(inplace=True)
df_join.rename(columns={'Neighbourhood_y':'Neighbourhood'}, inplace=True)
#df_join.head()
df=df_join

## Import Geocodes
### Doesn't work, not even the examples on the given Geocoder page.

In [39]:
#!pip install geocoder
import geocoder

In [76]:
postal_code='M3A'
borough='North York'

coordinates = None
    
str='Toronto,', borough
    
g = geocoder.google(str)
coordinates=g.latlng

In [77]:
print(g)
    

<[REQUEST_DENIED] Google - Geocode [empty]>


## Import CSV-File with Geodata

In [86]:
url='https://cocl.us/Geospatial_data'

df2=pd.read_csv(url)


In [88]:
df_total=pd.merge(df, df2, on='Postal Code')

In [89]:
df_total

Postal Code           Borough  \
0           M3A        North York   
1           M4A        North York   
2           M5A  Downtown Toronto   
3           M6A        North York   
4           M7A  Downtown Toronto   
..          ...               ...   
98          M8X         Etobicoke   
99          M4Y  Downtown Toronto   
100         M7Y      East Toronto   
101         M8Y         Etobicoke   
102         M8Z         Etobicoke   

                                         Neighbourhood   Latitude  Longitude  
0                                            Parkwoods  43.753259 -79.329656  
1                                     Victoria Village  43.725882 -79.315572  
2                            Regent Park, Harbourfront  43.654260 -79.360636  
3                     Lawrence Manor, Lawrence Heights  43.718518 -79.464763  
4          Queen's Park, Ontario Provincial Government  43.662301 -79.389494  
..                                                 ...        ...        ...  
98       The Kingsway, Montgomery Road, Old Mill North  43.653654 -79.506944  
99                                Church and Wellesley  43.665860 -79.383160  
100  Business reply mail Processing Centre, South C...  43.662744 -79.321558  
101  Old Mill South, King's Mill Park, Sunnylea, Hu...  43.636258 -79.498509  
102  Mimico NW, The Queensway West, South of Bloor,...  43.628841 -79.520999  

[103 rows x 5 columns]